In [1]:
from pyquil import Program, get_qc
from pyquil.gates import *
import numpy as np

In [2]:
qvm = get_qc('9q-generic-qvm')

### Bit Flip

We first explore a simple "bit flip" noise model -- with some probability, the qubit $\vert 0 \rangle$ flips to $\vert 1 \rangle$. In other words, we are modeling the channel<br>
<br>
$$ \rho \rightarrow p \left( X \rho X^{\dagger}\right) + (1-p) \left( I \rho I^{\dagger} \right)$$<br>
where $\rho = \vert 0 \rangle \langle 0 \vert$.

In [3]:
def kraus_ops_bit_flip(prob):
    """
    :param prob: probability of bit-flip
    :return: list of Kraus operators
    """
    # define flip (X) and not flip (I) Kraus operators
    I_ = np.sqrt(1 - prob) * np.array([[1, 0], [0, 1]])
    X_ = np.sqrt(prob) * np.array([[0, 1], [1, 0]])
    return [I_, X_]

In [4]:
# pick probability of bit flip, and num_shots
prob = 0.3
num_shots = 1000

# obtain Kraus operators associated with the channel
kraus_ops = kraus_ops_bit_flip(prob)

# create Program with noisy gates
p = Program()
p.declare("ro", "BIT", 1)
p.inst(X(0))
p.define_noisy_gate("X", [0], kraus_ops)
p.measure(0, "ro")
p.wrap_in_numshots_loop(shots=1000)

# obtain and print out the results
results = qvm.run(qvm.compile(p))
print ("Measured '0': ", len([i for i in results if i == [0]]), "/", num_shots, " times")
print ("Measured '1': ", len([i for i in results if i == [1]]), "/", num_shots, " times")

Measured '0':  0 / 1000  times
Measured '1':  1000 / 1000  times


### Amplitude Damping

Here, we model a noisy process where an excited $\vert 1 \rangle$ randomly decays to the $\vert 0 \rangle$ state with some probability. In other words, we model the channel<br>
<br>
$$ \rho \rightarrow \left( K_{1} \rho K_{1}^{\dagger}\right) + \left( K_{2} \rho K_{2}^{\dagger} \right)$$<br>
where $\rho = \vert 1 \rangle \langle 1 \vert$, and <br>
$$K_{1} = \left( 
\begin{array}{cc}
1 & 0 \\
0 & \sqrt{1-p}
\end{array}
\right) ,
\quad\quad\quad
K_{2} = \left( 
\begin{array}{cc}
0 & \sqrt{p} \\
0 & 0
\end{array}
\right) 
$$

In [5]:
def random_unitary():
    """
    :return: array of shape (2, 2) representing random unitary matrix drawn from Haar measure
    """
    # draw complex matrix from Ginibre ensemble
    z = np.random.randn(2, 2) + 1j * np.random.randn(2, 2)
    # QR decompose this complex matrix
    q, r = np.linalg.qr(z)
    # make this decomposition unique
    d = np.diagonal(r)
    l = np.diag(d) / np.abs(d)
    return np.matmul(q, l)

In [6]:
def kraus_ops_amp_damping(prob):
    """
    :param prob: probability of |1> to |0> decay
    :return: list of Kraus operators
    """
    # define imperfect identity (I_) and decay (D_) Kraus operators
    I_ = np.array([[1, 0], [0, np.sqrt(1 - prob)]])
    D_ = np.array([[0, np.sqrt(prob)], [0, 0]])
    return [I_, D_]

In [7]:
# pick probability of random decay, and num_shots
prob = 0.3
num_shots = 1000

# obtain Kraus operators associated with the channel
kraus_ops = kraus_ops_amp_damping(prob)

# create Program with noisy gates
p = Program()
p.declare("ro", "BIT", 1)
p.inst(X(0))
p.defgate("DummyGate", random_unitary())
p.inst(("DummyGate", 0))
p.define_noisy_gate("DummyGate", [0], kraus_ops)
p.measure(0, "ro")
p.wrap_in_numshots_loop(shots=1000)

# obtain and print out the results
results = qvm.run(qvm.compile(p))
print ("Measured '0': ", len([i for i in results if i == [0]]), "/", num_shots, " times")
print ("Measured '1': ", len([i for i in results if i == [1]]), "/", num_shots, " times")

Measured '0':  53 / 1000  times
Measured '1':  947 / 1000  times


### Dephasing

Here, we model a noisy process where the relative phase between $\vert 0 \rangle$ and $\vert 1 \rangle$ goes from $+1$ to $-1$ with some probability. We are essentially modeling the channel<br>
<br>
$$ \rho \rightarrow p \left( Z \rho Z^{\dagger} \right) + (1 - p) \left( I \rho I^{\dagger}\right)$$<br>
where $\rho = \vert + \rangle \langle + \vert$, s.t. $\vert + \rangle = (1/2) \left( \vert 0 \rangle + \vert 1 \rangle\right)$.

In [8]:
def kraus_ops_dephasing(prob):
    """
    :param prob: probability of applying Z operator
    :return: list of Kraus operators
    """
    # define probabilistic identity (I_) and Z (Z_) Kraus operators
    I_ = np.sqrt(1 - prob) * np.array([[1, 0], [0, 1]])
    Z_ = np.sqrt(prob) * np.array([[1, 0], [0, -1]])
    return [I_, Z_]

In [9]:
# pick probability of dephasing, along with num_shots
prob = 0.3
num_shots = 1000

# obtain Kraus operators associating with the dephasing channel
kraus_ops = kraus_ops_dephasing(prob)

# create Program with noisy gates
p = Program()
p.declare("ro", "BIT", 1)
p.inst(H(0))
p.defgate("DummyGate", random_unitary())
p.inst(("DummyGate", 0))
p.define_noisy_gate("DummyGate", [0], kraus_ops)
# measure in Hadamard basis to measure effect of dephasing
p.inst(H(0))
p.measure(0, "ro")
p.wrap_in_numshots_loop(shots=1000)

# obtain and print out the results
results = qvm.run(qvm.compile(p))
print ("Measured '0': ", len([i for i in results if i == [0]]), "/", num_shots, " times")
print ("Measured '1': ", len([i for i in results if i == [1]]), "/", num_shots, " times")

Measured '0':  974 / 1000  times
Measured '1':  26 / 1000  times
